# Converting our generated data to vcf so it can be used with GNOMIX and LAI-NET

In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
from my_knn_module.import_data import import_genome_data_as_df

In [2]:
prediction_filename = "WGAN.tsv"
ag_filename = "WGAN.hapt"
general_name = "WGAN"

## Pairing AGs
Going from haplotypes to genomes

In [3]:
df = pd.read_csv('./generated/' + prediction_filename, delimiter = ',')
df.head()

,Sample,Superpopulation code
0,AG0,AMR
1,AG1,EAS
2,AG2,EAS
3,AG3,EAS
4,AG4,AFR


In [4]:
paired_df = pd.DataFrame(data=None, columns=["Prev Index", "Sample", "Superpopulation code"])
for i in range(df.shape[0]):
    if not(i in set(paired_df["Prev Index"])):
        inserted = False
        for j in range(i+1, df.shape[0]):
            if not(j in set(paired_df["Prev Index"])) and df.iloc[j]["Superpopulation code"] == df.iloc[i]["Superpopulation code"]:
                paired_df = paired_df.append({"Prev Index": i, "Sample": df.iloc[i]["Sample"], "Superpopulation code": df.iloc[i]["Superpopulation code"]}, ignore_index=True)
                paired_df = paired_df.append({"Prev Index": j, "Sample": df.iloc[j]["Sample"], "Superpopulation code": df.iloc[j]["Superpopulation code"]}, ignore_index=True)
                inserted = True
                break
        if not(inserted): print("Couldn't find pair for row " + str(i))
    if i % 1000 == 0: print("..." + str(i) + "/"+ str(df.shape[0]))
paired_df.head()    

...0/5008
...1000/5008
...2000/5008
...3000/5008
...4000/5008
...5000/5008
Couldn't find pair for row 5001
Couldn't find pair for row 5007


,Prev Index,Sample,Superpopulation code
0,0,AG0,AMR
1,18,AG18,AMR
2,1,AG1,EAS
3,2,AG2,EAS
4,3,AG3,EAS


In [5]:
ag_df = pd.read_csv('./data/' + ag_filename, delimiter = ' ')
ag_df = ag_df.rename(columns={0: "Type", 1: "Sample"})
ag_df = ag_df.reindex(paired_df["Prev Index"])
ag_df.head()

,Type,Sample,0,1,2,3,4,5,6,7,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
Prev Index,,,,,,,,,,,,,,,,,,,,,
0,AG,AG0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
18,AG,AG18,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,AG,AG1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,AG,AG2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AG,AG3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
ag_df.to_csv("./generated/" + general_name + "_paired.hapt", sep=" ", index=False)

## Importing our data

In [7]:
# Using the paired by superpopulation .hapt because .vcf output requires 2 chromosomes 
ag_df = pd.read_csv("./generated/" + general_name + "_paired.hapt", delimiter = ' ')
ag_df.head()

,Type,Sample,0,1,2,3,4,5,6,7,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,AG,AG0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,AG,AG18,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,AG,AG1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,AG,AG2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AG,AG3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data_dim = ag_df.shape
print(ag_df.shape, "\n")
print("We can see that we've got {} SNIPs and {} haplotypes.".format(data_dim[1] - 2, data_dim[0]))

(5006, 10002) 

We can see that we've got 10000 SNIPs and 5006 haplotypes.


## First we create the **.sample** file
This file should contain (number of haplotype)/2 entries.
Here that number is 2503.

This file also holds our inferred ancestry information.

In [9]:
samples_df = pd.DataFrame(ag_df["Sample"].copy())
pop_df = pd.read_csv('./generated/' + prediction_filename, delimiter = ',')
#samples_df["population"] = pop_df.loc(pop_df["Sample"] == samples_df["Sample"])["Superpopulation code"]

samples_df = pd.merge(samples_df, pop_df, on='Sample')
samples_df["group"] = samples_df["Superpopulation code"]
samples_df["sex"] = 1 # Set it to 1 because we have no information on sex

samples_df.drop(columns=["Superpopulation code"], inplace=True)

samples_df = samples_df.drop(index=[i for i in range(1, data_dim[0], 2)]) # We drop half of the values because .vcf needs full genotypes and we are working with haplotypes 

samples_df.to_csv("./generated/" + general_name + ".sample", index=False, sep=" ")

print(samples_df.shape, "\n")
samples_df.head()

(2503, 3) 



,Sample,group,sex
0,AG0,AMR,1
2,AG1,EAS,1
4,AG3,EAS,1
6,AG4,AFR,1
8,AG6,EUR,1


## Then we convert our data to **.hap**

In [10]:
ag_df = ag_df.drop(columns=['Type', 'Sample'])
ag_df.head()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
ag_df = ag_df.to_numpy().transpose() # .hap is, simply put, the transpose of our .hapt data
ag_df = pd.DataFrame(ag_df)

ag_df.to_csv("./generated/" + general_name + ".hap", header=False, index=False, sep=" ")

print(ag_df.shape, "\n")
ag_df.head()

(10000, 5006) 



,0,1,2,3,4,5,6,7,8,9,...,4996,4997,4998,4999,5000,5001,5002,5003,5004,5005
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Finaly we create our **.legend** file
It's created from an existing vcf files of real data used to train our model that was converted into a **.hap** + **.legend** using *bcftools*.
This file needs as many entries as there are SNIPs.

    ./bcftools convert --haplegendsample

In [12]:
label_df = pd.read_csv('./data/10k.hap', delimiter = ' ', header=None)
label_df = label_df.drop(columns=[i for i in range(5, 5013)])
label_df.rename(columns={0: 'Chromo num', 1: 'id', 2: 'position', 3: 'a0', 4: 'a1'}, inplace=True)
label_df.drop(columns=["Chromo num"], inplace=True)

label_df.to_csv("./generated/" + general_name + ".legend", index=False, sep=" ")

print(label_df.shape, "\n")
label_df.head()

(10000, 4) 



,id,position,a0,a1
0,15:27379578_C_A,27379578,C,A
1,15:27379592_T_A,27379592,T,A
2,15:27379787_C_T,27379787,C,T
3,15:27379947_C_T,27379947,C,T
4,15:27380842_A_G,27380842,A,G


## With all of our files
We can then use bcftools to convert these three files into a **.vcf** file.

To achieve this we use the following command from *bcftools*:

    ./bcftools convert --haplegendsample2vcf [.hap],[.legend],[.sample] -o [.vcf]

## Creating **.smap**
This will be used by the model

In [15]:
smap_df = pd.DataFrame(pd.read_csv('./data/' + ag_filename, delimiter = ' ')["Sample"].copy())
pop_df = pd.read_csv('./generated/' + prediction_filename, delimiter = ',')
smap_df = pd.merge(smap_df, pop_df, on='Sample')
smap_df = smap_df.drop(index=[i for i in range(1, smap_df.shape[0], 2)]) # We drop half of the values because .vcf needs full genotypes and we are working with haplotypes 

print(smap_df.shape)
smap_df.head()

(2504, 2)


,Sample,Superpopulation code
0,AG0,AMR
2,AG2,EAS
4,AG4,AFR
6,AG6,EUR
8,AG8,AFR


In [16]:
smap_df.to_csv("./generated/" + general_name + ".smap", sep="	", index=False, header=False)